## QUESTION ANSWERING USANDO EL MODELO DE ROBERT Y SQAC PARA EL EPAÑOL

La respuesta a preguntas es una tarea común de PNL con varias variantes. En algunas variantes, la tarea es de opción múltiple: se proporciona una lista de posibles respuestas con cada pregunta, y el modelo simplemente necesita devolver una distribución de probabilidad sobre las opciones. Una variante más desafiante de responder preguntas, que es más aplicable a las tareas de la vida real, es cuando no se brindan las opciones. En cambio, el modelo recibe un documento de entrada, llamado contexto, y una pregunta sobre el documento, y debe extraer la extensión del texto en el documento que contiene la respuesta. En este caso, el modelo no calcula una distribución de probabilidad sobre las respuestas, sino dos distribuciones de probabilidad sobre los tokens en el texto del documento, que representan el inicio y el final del lapso que contiene la respuesta. Esta variante se denomina "respuesta extractiva de preguntas".

La respuesta extractiva a preguntas es una tarea de PNL muy desafiante, y el tamaño del conjunto de datos requerido para entrenar un modelo de este tipo desde cero cuando las preguntas y respuestas son en lenguaje natural es prohibitivamente enorme. Como resultado, la respuesta a preguntas (como casi todas las tareas de PNL) se beneficia enormemente al comenzar con un sólido modelo de base preentrenado: comenzar con un sólido modelo de lenguaje preentrenado puede reducir el tamaño del conjunto de datos requerido para alcanzar una precisión determinada en varios órdenes de magnitud, lo que le permite para alcanzar un rendimiento muy sólido con conjuntos de datos sorprendentemente razonables.

Sin embargo, comenzar con un modelo previamente entrenado agrega dificultades: ¿de dónde obtiene el modelo? ¿Cómo se asegura de que sus datos de entrada se preprocesen y tokenicen de la misma manera que el modelo original? ¿Cómo modifica el modelo para agregar un cabezal de salida que coincida con su tarea de interés?

En este ejemplo, le mostraremos cómo cargar un modelo de la biblioteca Hugging Face 🤗Transformers para enfrentar este desafío. También cargaremos un conjunto de datos de respuesta a preguntas de referencia de la biblioteca 🤗Datasets : este es otro repositorio de código abierto que contiene una amplia gama de conjuntos de datos en muchas modalidades, desde NLP hasta visión y más. Tenga en cuenta, sin embargo, que no existe ningún requisito de que estas bibliotecas deban usarse entre sí. Si desea entrenar un modelo de 🤗Transformers con sus propios datos, o si desea cargar datos de 🤗 Conjuntos de datos y entrenar sus propios modelos completamente no relacionados con ellos, eso es posible (¡y muy recomendable!)

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install pytorch-lightning


Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 12.2 MB/s eta 0:03:41
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:47
     ---------------------------------------- 0.0/2.7 GB 11.8 MB/s eta 0:03:48
     ---------------------------------------- 0.0/2.7 GB 12.0 MB/s eta 0:03:45
     ---------------------------------------- 0.0/2.7 GB 12.0 MB/s eta 0:03:44
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:45
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:46
     ---------------------------------------- 0.0/2.7 GB 11.8 MB/s eta 0:03:47
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:45
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:45
     ---------------------------------------- 0.0/2.7 GB 11.9 MB/s eta 0:03:45
 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\Usuario\AppData\Local\Temp\pip-req-build-9t1wuyvr'



  Cloning https://github.com/huggingface/transformers.git to c:\users\usuario\appdata\local\temp\pip-req-build-9t1wuyvr
  Resolved https://github.com/huggingface/transformers.git to commit 37ea04013b34b39c01b51aeaacd8d56f2c62a7eb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9969819 sha256=4ed492a4beb24a63e8304431e81e6165ffb0209c1d21528133215ee64fb31a39
  Stored in directory: C:\Users\Usuario\AppData\Local\Temp\pip-ephem-wheel-cache-7r6xrttm\wheels\32\4b\78\f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transforme

In [1]:
# PRINCIPAL LIBRARIES
import torch

import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:50"
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # specify which GPU(s) to be used
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print(torch.cuda.is_available())
print(torch.cuda.device_count())  

True
1


In [3]:
#from datasets import load_dataset

#datasets = load_dataset("squad")

In [2]:
from datasets import load_dataset
#datasets = load_dataset('squad_es/v1.1.0')

#datasets = load_dataset('squad_es', 'v1.1.0')

c:\Users\Usuario\Desktop\univalle\tesis\QA-roberta_2\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

datasets = load_dataset('SQAC.py',trust_remote_code=True)

In [4]:
print(datasets["train"][0])

{'id': '6cf3dcd6-b5a3-4516-8f9e-c5c1c6b66628', 'title': 'Historia de Japón', 'context': 'La historia de Japón (日本の歴史 o 日本史, Nihon no rekishi / Nihonshi?) es la sucesión de hechos acontecidos dentro del archipiélago japonés. Algunos de estos hechos aparecen aislados e influenciados por la naturaleza geográfica de Japón como nación insular, en tanto que otra serie de hechos, obedece a influencias foráneas como en el caso del Imperio chino, el cual definió su idioma, su escritura y, también, su cultura política. Asimismo, otra de las influencias foráneas fue la de origen occidental, lo que convirtió al país en una nación industrial, ejerciendo con ello una esfera de influencia y una expansión territorial sobre el área del Pacífico. No obstante, dicho expansionismo se detuvo tras la Segunda Guerra Mundial y el país se posicionó en un esquema de nación industrial con vínculos a su tradición cultural.', 'question': '¿Qué influencia convirtió Japón en una nación industrial?', 'answers': {'tex

In [5]:



# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
#model_checkpoint = "PlanTL-GOB-ES/roberta-base-bne-sqac"
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-sqac")
#model = AutoModelForQuestionAnswering.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-sqac")


In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)

In [8]:
train_set = tokenized_datasets["train"].with_format("numpy")[
    :
]  # Load the whole dataset as a dict of numpy arrays
validation_set = tokenized_datasets["validation"].with_format("numpy")[:]

# Afinando el modelo
¡Eso fue mucho trabajo! Pero ahora que nuestros datos están listos, todo funcionará sin problemas. Primero, descargamos el modelo preentrenado y lo ajustamos. Dado que nuestra tarea es responder preguntas, usamos la TFAutoModelForQuestionAnsweringclase. Al igual que con el tokenizador, el from_pretrained()método descargará y almacenará en caché el modelo por nosotros:

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [10]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,TrainingArguments, Trainer

#model = AutoModelForQuestionAnswering.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-sqac")
#model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint,from_pt=True)
#model=RoBertaForQuestionAnswering.pretrained("PlanTL-GOB-ES/roberta-base-bne-sqac")
#model = AutoModelForQuestionAnswering.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-sqac")
model = AutoModelForQuestionAnswering.from_pretrained("PlanTL-GOB-ES/roberta-large-bne-sqac")

c:\Users\Usuario\Desktop\univalle\tesis\QA-roberta_2\.conda\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--PlanTL-GOB-ES--roberta-large-bne-sqac. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


La advertencia nos dice que estamos desechando algunos pesos e inicializando otros. ¡No entrar en pánico! Esto es absolutamente normal. Recuerde que modelos como BERT y Distilbert están capacitados en una tarea de modelado de lenguaje , pero estamos cargando el modelo como un TFAutoModelForQuestionAnswering, lo que significa que queremos que el modelo realice una tarea de respuesta a preguntas . Este cambio requiere que se elimine la capa de salida final o el "cabezal" y se reemplace con un nuevo cabezal adecuado para la nueva tarea. El from_pretrained método manejará todo esto por nosotros, y la advertencia está ahí simplemente para recordarnos que se ha realizado una cirugía de modelo y que el modelo no generará predicciones útiles hasta que las capas recién inicializadas se hayan ajustado en algunos datos. .

A continuación, podemos crear un optimizador y especificar una función de pérdida. Por lo general, puede obtener un rendimiento ligeramente mejor utilizando la disminución de la tasa de aprendizaje y la disminución del peso desacoplado, pero para los fines de este ejemplo, el Adamoptimizador estándar funcionará bien. Tenga en cuenta, sin embargo, que al ajustar un modelo de transformador preentrenado, ¡generalmente querrá usar una tasa de aprendizaje baja! Encontramos que los mejores resultados se obtienen con valores en el rango de 1e-5 a 1e-4, y el entrenamiento puede divergir completamente en la tasa de aprendizaje predeterminada de Adam de 1e-3.

In [ ]:
from huggingface_hub import login, HfApi


# Login to Hugging Face
# Replace 'your_token' with your actual Hugging Face token
hf_token = input("Enter your Hugging Face token: ")
login(token=hf_token)

In [11]:



training_args = TrainingArguments(
    output_dir="roberta-large-QA",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
   push_to_hub=True,
)

Y ahora solo compilamos y ajustamos el modelo. Para su comodidad, todos los modelos de 🤗 Transformers vienen con una pérdida predeterminada que coincide con su cabeza de salida, aunque, por supuesto, puede usar la suya propia. Debido a que la pérdida incorporada se calcula internamente durante el paso hacia adelante, al usarla, es posible que algunas métricas de Keras se comporten mal o den resultados inesperados. Sin embargo, esta es un área de desarrollo muy activo en 🤗 Transformers, ¡así que esperamos tener una buena solución para ese problema pronto!

Por ahora, sin embargo, usemos la pérdida integrada sin ninguna métrica. Para obtener la pérdida incorporada, simplemente omita el lossargumento de compile.

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5928\3014769959.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Y ahora podemos entrenar nuestro modelo. Tenga en cuenta que no estamos pasando etiquetas separadas: las etiquetas son claves en el dictado de entrada, para que sean visibles para el modelo durante el pase hacia adelante para que pueda calcular la pérdida incorporada.

In [13]:
trainer.train()


  0%|          | 0/940 [00:00<?, ?it/s]c:\Users\Usuario\Desktop\univalle\tesis\QA-roberta_2\.conda\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 53%|█████▎    | 500/940 [6:48:47<6:06:29, 49.98s/it] 

{'loss': 3.5232, 'grad_norm': 22.549571990966797, 'learning_rate': 9.361702127659576e-06, 'epoch': 0.53}


                                                      
100%|██████████| 940/940 [13:38:35<00:00, 52.25s/it]

{'eval_loss': 2.7141852378845215, 'eval_runtime': 2331.625, 'eval_samples_per_second': 0.819, 'eval_steps_per_second': 0.051, 'epoch': 1.0}
{'train_runtime': 49115.9385, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.019, 'train_loss': 3.1862339750249333, 'epoch': 1.0}


TrainOutput(global_step=940, training_loss=3.1862339750249333, metrics={'train_runtime': 49115.9385, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.019, 'total_flos': 1.0473028750227456e+16, 'train_loss': 3.1862339750249333, 'epoch': 1.0})

In [14]:
trainer.model.save_pretrained("roberta-large-QA")

In [23]:


from transformers import pipeline

question_answerer = pipeline("question-answering", model="roberta-large-QA")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [24]:
#test question answering

context = """  La Comisión de la Verdad en Colombia, creada en el marco del Acuerdo de Paz de 2016, ha adelantado un arduo trabajo para esclarecer los hechos y patrones de graves violaciones a los derechos humanos y del Derecho Internacional Humanitario ocurridos durante el conflicto armado en el país. Entre sus principales labores se destacan:

Recepción de testimonios: La Comisión ha recibido más de 14.000 testimonios de víctimas, victimarios, agentes del Estado y otros actores del conflicto, recopilando un acervo invaluable de relatos que permiten comprender la complejidad del conflicto y sus múltiples dimensiones.

Investigación: A partir de los testimonios y otras fuentes de información, la Comisión ha realizado investigaciones exhaustivas sobre diferentes temáticas, incluyendo masacres, desapariciones forzadas, desplazamiento forzado, reclutamiento de menores, violencia sexual, entre otras.

Elaboración del Informe Final: La Comisión se encuentra en la fase final de elaboración de su Informe Final, en el que se presentarán los hallazgos de sus investigaciones, un análisis del conflicto armado y recomendaciones para la no repetición. El Informe Final será presentado al país el 28 de junio de 2024.

Contribución a la construcción de paz: La Comisión ha desarrollado diversas iniciativas para contribuir a la construcción de paz en Colombia, incluyendo programas de pedagogía para la paz, promoción de la convivencia y la reconciliación, y apoyo a las víctimas en su proceso de reparación.
"""
question = "¿cuales son las principales funciones de la comisión de la Verdad en Colombia?"
input_question = input("Enter your question: ")


if input_question:
    question = input_question
print("question: ",question)
question_answerer(question=question, context=context)

question:  ¿cuales son las principales funciones de la comisión de la Verdad en Colombia?


{'score': 0.11679749190807343,
 'start': 333,
 'end': 359,
 'answer': '\n\nRecepción de testimonios'}

¡Y eso es! Recuerde que este ejemplo fue diseñado para ser rápido de ejecutar en lugar de avanzado, y el modelo entrenado aquí ciertamente cometerá errores. Si usa un modelo más grande para basar su entrenamiento y se toma el tiempo para ajustar los hiperparámetros adecuadamente, encontrará que puede lograr pérdidas mucho mejores (y, en consecuencia, respuestas más precisas).

Finalmente, puede enviar el modelo al HuggingFace Hub. Al empujar este modelo tendrás:

Una bonita tarjeta modelo generada para usted que contiene hiperparámetros y métricas del entrenamiento del modelo,
Una API web para llamadas de inferencia,
Un widget en la página del modelo que permite a otros probar su modelo. Este modelo está actualmente alojado aquí y hemos preparado una interfaz de usuario ordenada separada para usted aquí .